In [8]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import json

In [4]:
urll = 'https://kosis.kr/openapi/statisticsData.do?method=getList&apiKey=Yjg5YjZmNTU0OThkMWNmYTE3MDY0ZWQzZjUyNjQ0YzY=&format=json&jsonVD=Y&userStatsId=rsh1502/110/TX_11007_A065/2/1/20210714110402&prdSe=Y&newEstPrdCnt=1'

In [5]:
dir_for_shinhan = 'C:/Users/shic/Desktop/shinhan_simon/chromedriver/chromedriver.exe'
dir_for_seunghun = 'C:/Users/rsh15/Desktop/seunghuni/selenium/chromedriver.exe'

In [6]:
driver = webdriver.Chrome(dir_for_shinhan)

In [7]:
driver.get(urll)

In [10]:
a = BeautifulSoup(driver.page_source,'html.parser')
a = a.find_all('body')[0].text

In [14]:
tmp_df = pd.DataFrame(json.loads(a))
tmp_df

,TBL_NM,PRD_DE,TBL_ID,ITM_NM,ITM_ID,UNIT_NM,ORG_ID,UNIT_NM_ENG,C1_OBJ_NM,C2_OBJ_NM,DT,C2,PRD_SE,C1,C1_NM,C2_NM
0,1-11. 경기도,2019,TX_11007_A065,경기도,16110ABC9,천원,110,Thousand Won,시군구별,세목별,3254167190,15110AD6ABAB,A,15110AR000,합계,재산세
1,1-11. 경기도,2019,TX_11007_A065,경기도,16110ABC9,천원,110,Thousand Won,시군구별,세목별,56694004,15110AD6BAAD,A,15110AR000,합계,재산세
2,1-11. 경기도,2019,TX_11007_A065,경기도,16110ABC9,천원,110,1000 won,시군구별,세목별,0,15110AD6ABAB,A,15110AR001,경기도,재산세
3,1-11. 경기도,2019,TX_11007_A065,경기도,16110ABC9,천원,110,1000 won,시군구별,세목별,0,15110AD6BAAD,A,15110AR001,경기도,재산세
4,1-11. 경기도,2019,TX_11007_A065,경기도,16110ABC9,천원,110,Thousand Won,시군구별,세목별,86509782,15110AD6ABAB,A,15110AR0BB,광주시,재산세
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,1-11. 경기도,2019,TX_11007_A065,경기도,16110ABC9,천원,110,Thousand Won,시군구별,세목별,0,15110AD6BAAD,A,15110AR0Bu,과천시,재산세
62,1-11. 경기도,2019,TX_11007_A065,경기도,16110ABC9,천원,110,Thousand Won,시군구별,세목별,0,15110AD6ABAB,A,15110AR0Bx,가평군,재산세
63,1-11. 경기도,2019,TX_11007_A065,경기도,16110ABC9,천원,110,Thousand Won,시군구별,세목별,24445614,15110AD6BAAD,A,15110AR0Bx,가평군,재산세
64,1-11. 경기도,2019,TX_11007_A065,경기도,16110ABC9,천원,110,Thousand Won,시군구별,세목별,0,15110AD6ABAB,A,15110AR0Bz,연천군,재산세


In [15]:
tmp_df['C1_NM'].unique()

array(['합계', '경기도', '광주시', '수원시', '성남시', '고양시', '부천시', '안양시', '안산시',
       '용인시', '의정부시', '남양주시', '평택시', '광명시', '시흥시', '군포시', '화성시', '파주시',
       '이천시', '구리시', '김포시', '포천시', '안성시', '하남시', '의왕시', '양주시', '오산시',
       '여주시', '양평군', '동두천시', '과천시', '가평군', '연천군'], dtype=object)

In [29]:
tmp_df['DT'] = pd.to_numeric(tmp_df['DT'])
tmp_df

,TBL_NM,PRD_DE,TBL_ID,ITM_NM,ITM_ID,UNIT_NM,ORG_ID,UNIT_NM_ENG,C1_OBJ_NM,C2_OBJ_NM,DT,C2,PRD_SE,C1,C1_NM,C2_NM
0,1-11. 경기도,2019,TX_11007_A065,경기도,16110ABC9,천원,110,Thousand Won,시군구별,세목별,3254167190,15110AD6ABAB,A,15110AR000,합계,재산세
1,1-11. 경기도,2019,TX_11007_A065,경기도,16110ABC9,천원,110,Thousand Won,시군구별,세목별,56694004,15110AD6BAAD,A,15110AR000,합계,재산세
2,1-11. 경기도,2019,TX_11007_A065,경기도,16110ABC9,천원,110,1000 won,시군구별,세목별,0,15110AD6ABAB,A,15110AR001,경기도,재산세
3,1-11. 경기도,2019,TX_11007_A065,경기도,16110ABC9,천원,110,1000 won,시군구별,세목별,0,15110AD6BAAD,A,15110AR001,경기도,재산세
4,1-11. 경기도,2019,TX_11007_A065,경기도,16110ABC9,천원,110,Thousand Won,시군구별,세목별,86509782,15110AD6ABAB,A,15110AR0BB,광주시,재산세
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,1-11. 경기도,2019,TX_11007_A065,경기도,16110ABC9,천원,110,Thousand Won,시군구별,세목별,0,15110AD6BAAD,A,15110AR0Bu,과천시,재산세
62,1-11. 경기도,2019,TX_11007_A065,경기도,16110ABC9,천원,110,Thousand Won,시군구별,세목별,0,15110AD6ABAB,A,15110AR0Bx,가평군,재산세
63,1-11. 경기도,2019,TX_11007_A065,경기도,16110ABC9,천원,110,Thousand Won,시군구별,세목별,24445614,15110AD6BAAD,A,15110AR0Bx,가평군,재산세
64,1-11. 경기도,2019,TX_11007_A065,경기도,16110ABC9,천원,110,Thousand Won,시군구별,세목별,0,15110AD6ABAB,A,15110AR0Bz,연천군,재산세


In [43]:
df_a = tmp_df.loc[(tmp_df['DT']!=0) & (tmp_df['C1_NM']!='합계'),['ITM_NM','DT','C1_NM','C2_NM']]
df_a.reset_index(drop=True)

,ITM_NM,DT,C1_NM,C2_NM
0,경기도,86509782,광주시,재산세
1,경기도,239628167,수원시,재산세
2,경기도,321964320,성남시,재산세
3,경기도,232091686,고양시,재산세
4,경기도,145624807,부천시,재산세
5,경기도,128016660,안양시,재산세
6,경기도,131467630,안산시,재산세
7,경기도,322755507,용인시,재산세
8,경기도,64901062,의정부시,재산세
9,경기도,137908529,남양주시,재산세


In [44]:
df_b = tmp_df.groupby(['ITM_NM','C1_NM','C2_NM'],as_index=False)['DT'].sum()
df_b.reset_index(drop=True)

,ITM_NM,C1_NM,C2_NM,DT
0,경기도,가평군,재산세,24445614
1,경기도,경기도,재산세,0
2,경기도,고양시,재산세,232091686
3,경기도,과천시,재산세,34611621
4,경기도,광명시,재산세,69811997
5,경기도,광주시,재산세,86509782
6,경기도,구리시,재산세,40890323
7,경기도,군포시,재산세,53353654
8,경기도,김포시,재산세,117827044
9,경기도,남양주시,재산세,137908529
